In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Load the image
image_path = "golf.jpg"  # Replace with your image path
image = cv2.imread('D:\Humaira\Logiqon\pocket\golf2.jpg')

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Convert the image to RGB as MediaPipe uses RGB format
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Perform pose detection
results = pose.process(image_rgb)

# Draw the pose landmarks on the original image
mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Check if pose landmarks were detected
if results.pose_landmarks:
    landmarks = results.pose_landmarks.landmark
    
    # Calculate angles for right side of the body
    
    # Shoulder, Elbow, Wrist (Elbow Angle)
    shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
             landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
             landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    elbow_angle = calculate_angle(shoulder, elbow, wrist)
    
    # Hand Position (Wrist, Elbow, Shoulder)
    hand_position_angle = calculate_angle(wrist, elbow, shoulder)
    
    # Shaft Angle (Wrist, Elbow, Ground level: approximated by the horizontal axis)
    shaft_angle = calculate_angle([wrist[0], 1], wrist, elbow)
    
    # Knee Angle (Hip, Knee, Ankle)
    hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
           landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
             landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    knee_angle = calculate_angle(hip, knee, ankle)
    
    print(f"Elbow Angle: {elbow_angle} degrees")
    print(f"Hand Position Angle: {hand_position_angle} degrees")
    print(f"Shaft Angle: {shaft_angle} degrees")
    print(f"Knee Angle: {knee_angle} degrees")

# Estimate accuracy based on the angles
def estimate_overall_accuracy(elbow_angle, hand_position_angle, shaft_angle, knee_angle):
    # Define ideal ranges (example values)
    ideal_elbow_angle = 90
    ideal_hand_position_angle = 180  # fully extended arms
    ideal_shaft_angle = 45
    ideal_knee_angle = 150
    
    # Calculate accuracy scores for each
    elbow_accuracy = 100 - abs(ideal_elbow_angle - elbow_angle) / ideal_elbow_angle * 100
    hand_position_accuracy = 100 - abs(ideal_hand_position_angle - hand_position_angle) / ideal_hand_position_angle * 100
    shaft_accuracy = 100 - abs(ideal_shaft_angle - shaft_angle) / ideal_shaft_angle * 100
    knee_accuracy = 100 - abs(ideal_knee_angle - knee_angle) / ideal_knee_angle * 100
    
    # Calculate overall accuracy as an average of individual accuracies
    overall_accuracy = (elbow_accuracy + hand_position_accuracy + shaft_accuracy + knee_accuracy) / 4
    return overall_accuracy

overall_accuracy = estimate_overall_accuracy(elbow_angle, hand_position_angle, shaft_angle, knee_angle)
print(f"Estimated Overall Swing Accuracy: {overall_accuracy:.2f}%")

# Create a blank image for text
height, width, _ = image.shape
text_height = 200  # Height of the text section
text_image = np.zeros((text_height, width, 3), dtype=np.uint8)
text_image.fill(255)  # White background

# Add text to the text image
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(text_image, f'Elbow Angle: {int(elbow_angle)} deg', (20, 40), 
            font, 1, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(text_image, f'Hand Position Angle: {int(hand_position_angle)} deg', (20, 80), 
            font, 1, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(text_image, f'Shaft Angle: {int(shaft_angle)} deg', (20, 120), 
            font, 1, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(text_image, f'Knee Angle: {int(knee_angle)} deg', (20, 160), 
            font, 1, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(text_image, f'Accuracy: {overall_accuracy:.2f}%', (20, 200), 
            font, 1, (0, 0, 0), 2, cv2.LINE_AA)

# Concatenate the original image with the text image
result_image = np.vstack((image, text_image))

# Save the concatenated image
output_path = "golf_swing_analysis_extended.jpg"  # Specify the output file path
cv2.imwrite(output_path, result_image)

print(f"Image saved as {output_path}")


Elbow Angle: 64.47780608860397 degrees
Hand Position Angle: 64.47780608860397 degrees
Shaft Angle: 36.69832833011635 degrees
Knee Angle: 174.02155714050886 degrees
Estimated Overall Swing Accuracy: 68.25%
Image saved as golf_swing_analysis_extended.jpg
